In [144]:
#Import modules
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [133]:
%load_ext google.cloud.bigquery

In [134]:
%%bigquery mydata
SELECT * FROM `skyuk-uk-nowtv-analytics-prod.SWA63.Page_transitions_for_modelling_SUMMARY`


In [135]:
mydata.head()


In [136]:
# What usernames are there?
mydata['username'].drop_duplicates()

In [137]:
# Create a dataframe for each individual username
for i in mydata['username'].drop_duplicates().tolist():
    exec ("df_{0}=mydata[mydata['username']=='{0}']".format(i))

In [149]:
# Select one of the dataframes to work from
myworkingdf=df_Roku

#number of connections
print(len(myworkingdf))

#Sort
myworkingdf["N"] = pd.to_numeric(myworkingdf["N"])
myworkingdf=myworkingdf.sort_values('pc_navs_overall',ascending=False)
myworkingdf['pc_sum']=myworkingdf['pc_navs_overall'].cumsum()

#myworkingdf=myworkingdf[myworkingdf['N']>100]

#Only select top 90% of navigations
myworkingdf=myworkingdf[myworkingdf['pc_sum']<0.80]

print(len(myworkingdf))
myworkingdf.head(30)

In [150]:
#Create empty directed graph
mygraph=nx.DiGraph()

In [151]:
#Add pages navigated from or two as nodes
pages_from = myworkingdf['page_summary'].tolist()
pages_to = myworkingdf['next_page'].tolist()
pages=list(set(pages_from+pages_to))
len(pages)
mygraph.add_nodes_from(pages)

#Add pages navigated from or two as nodes
#pages_from = myworkingdf['page_summary'].tolist()
#pages_to = myworkingdf['next_page'].tolist()
#pages=list(set(pages_from+pages_to))
#len(pages)
#for n,i in enumerate(pages):
#    print(n)
#    print(i)
#    mygraph.add_node(n,label=i)


In [152]:
#add edges with weights
myworkingdf['navigation'] = myworkingdf[['page_summary', 'next_page','pc_navs_from_page']].apply(tuple, axis=1)
myworkingdf.head()
navigations=myworkingdf['navigation'].tolist()

for i in navigations:
    mygraph.add_edge(i[0],i[1],weight=i[2])

In [153]:
mygraph.nodes()
mygraph.edges(data=True)


In [148]:
#More precise drawing options

#Position the nodes on the graph by the centrality metrics of the node
dc=nx.degree_centrality(mygraph)
bc=nx.betweenness_centrality(mygraph)
pos={}
for i in bc:
    pos[i]=np.asarray([bc[i],dc[i]])
    
#Position the nodes on the graph using one of the defaults
#pos = nx.spectral_layout(mygraph)  # positions for all nodes
print(pos)

#Draw nodes at specified positions
nx.draw_networkx_nodes(mygraph, pos, node_size=2)

# edges
#nx.draw_networkx_edges(mygraph,pos)
#major edges only

#create a list of all weights
all_weights=[]
for (node1,node2,data) in mygraph.edges(data=True):
    all_weights.append(data['weight']) #we'll use this when determining edge thickness
#Dedupe to get list of unique weights
    unique_weights = list(set(all_weights))
#Plot the edges - one by one!
    for weight in unique_weights:
        # Loop through each unique weight, store the edges which have that weight in "weighted edges" and then plot with the corresponding width
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in mygraph.edges(data=True) if edge_attr['weight']==weight]
        width = weight*2
        nx.draw_networkx_edges(mygraph,pos,edgelist=weighted_edges,width=width)


# labels
nx.draw_networkx_labels(mygraph, pos, font_size=7, font_family='sans-serif')
plt.show()

In [47]:
#### Simple version without configuration
#Add pages navigated from or two as nodes
pages_from = myworkingdf['page_summary'].tolist()
pages_to = myworkingdf['next_page'].tolist()
pages=list(set(pages_from+pages_to))
len(pages)
mygraph.add_nodes_from(pages)

#add edges
myworkingdf['navigation'] = myworkingdf[['page_summary', 'next_page']].apply(tuple, axis=1)
myworkingdf.head()
navigations=myworkingdf['navigation'].tolist()
mygraph.add_edges_from(navigations)

# Draw
nx.draw(mygraph)
import matplotlib.pyplot as plt
plt.show()


In [179]:
#from itertools import combinations
#for n1, n2 in combinations(mygraph.nodes(),2):
#    print(n1,"-->",n2)


In [180]:
# Select one of the dataframes to work from
myworkingdf=df_Roku[df_Roku['next_page']=='PLAYER']



In [181]:
#Create empty directed graph
playergraph=nx.DiGraph()

In [182]:
#Add pages navigated from or two as nodes
pages_from = myworkingdf['page_summary'].tolist()
pages_to = myworkingdf['next_page'].tolist()
pages=list(set(pages_from+pages_to))
len(pages)
playergraph.add_nodes_from(pages)

#Add pages navigated from or two as nodes
#pages_from = myworkingdf['page_summary'].tolist()
#pages_to = myworkingdf['next_page'].tolist()
#pages=list(set(pages_from+pages_to))
#len(pages)
#for n,i in enumerate(pages):
#    print(n)
#    print(i)
#    mygraph.add_node(n,label=i)


In [183]:
#add edges with weights
myworkingdf['navigation'] = myworkingdf[['page_summary', 'next_page','pc_navs_from_page']].apply(tuple, axis=1)
myworkingdf.head()
navigations=myworkingdf['navigation'].tolist()

for i in navigations:
    playergraph.add_edge(i[0],i[1],weight=i[2])

In [184]:
#More precise drawing options
    
#Position the nodes on the graph using one of the defaults
pos = nx.circular_layout(mygraph)  # positions for all nodes
#print(pos)

#Draw nodes at specified positions
nx.draw_networkx_nodes(playergraph, pos, node_size=2)

# edges
#nx.draw_networkx_edges(mygraph,pos)
#major edges only

#create a list of all weights
all_weights=[]
for (node1,node2,data) in playergraph.edges(data=True):
    all_weights.append(data['weight']) #we'll use this when determining edge thickness
#Dedupe to get list of unique weights
    unique_weights = list(set(all_weights))
#Plot the edges - one by one!
    for weight in unique_weights:
        # Loop through each unique weight, store the edges which have that weight in "weighted edges" and then plot with the corresponding width
        weighted_edges = [(node1,node2) for (node1,node2,edge_attr) in playergraph.edges(data=True) if edge_attr['weight']==weight]
        width = weight*2
        nx.draw_networkx_edges(playergraph,pos,edgelist=weighted_edges,width=width)


# labels
nx.draw_networkx_labels(playergraph, pos, font_size=7, font_family='sans-serif')
plt.show()